# Agente para Detecção de Vulnerabilidades em Arquiteturas

Abaixo está um plano técnico conciso + exemplo mínimo de implementação (FastAPI) que satisfaz o desafio: receber imagem de diagrama arquitetural, extrair informação visual/textual, aplicar prompt engineering e gerar análise STRIDE estruturada via Azure OpenAI. Incluo dependências, arquitetura e um prompt template prático. Mantive o código reduzido — pronto para expandir conforme seu pipeline.

1 — Arquitetura proposta (resumo)

FastAPI — endpoint POST /analyze recebe multipart/form-data com imagem.

Azure AI Vision (Computer Vision) — (a) OCR / Read API para extrair texto do diagrama; (b) Image Analysis (caption, tags, objects) para obter descrição e entidades visuais.

Preprocessing / Prompt engineering — combinar: (i) legenda/caption, (ii) texto OCR, (iii) lista de objetos/tags detectados, (iv) heurísticas (trust boundaries, componentes identificados) em um prompt template estruturado.

Azure OpenAI — enviar prompt (Responses / Chat completions) pedindo análise STRIDE por componente e saída em JSON estruturado.

Resposta — FastAPI retorna JSON com: metadados da imagem, itens OCR, itens visuais, prompt usado e análise STRIDE (por componente + recomendações de mitigação).

2 — Dependências (mínimo)

fastapi, uvicorn, python-multipart

requests (ou azure-ai-vision e azure-ai-openai se preferir SDKs oficiais)

Pillow (manipulação de imagem)
(Recomendo usar SDKs oficiais Azure para produção.)

3 — Exemplo mínimo (FastAPI) — esqueleto funcional

In [ ]:
# requirements:
# pip install fastapi uvicorn python-multipart requests Pillow

import os, json, io
from fastapi import FastAPI, File, UploadFile, HTTPException
from pydantic import BaseModel
import requests
from PIL import Image

app = FastAPI()

# ENV vars (configurar antes): AZ_VISION_ENDPOINT, AZ_VISION_KEY, AZ_OPENAI_ENDPOINT, AZ_OPENAI_KEY, AZ_OPENAI_DEPLOYMENT
AZ_VISION_ENDPOINT = os.getenv("AZ_VISION_ENDPOINT")
AZ_VISION_KEY = os.getenv("AZ_VISION_KEY")
AZ_OPENAI_ENDPOINT = os.getenv("AZ_OPENAI_ENDPOINT")  # ex: https://<resource>.openai.azure.com/
AZ_OPENAI_KEY = os.getenv("AZ_OPENAI_KEY")
AZ_OPENAI_DEPLOYMENT = os.getenv("AZ_OPENAI_DEPLOYMENT")  # deployment name

if not all([AZ_VISION_ENDPOINT, AZ_VISION_KEY, AZ_OPENAI_ENDPOINT, AZ_OPENAI_KEY, AZ_OPENAI_DEPLOYMENT]):
    raise RuntimeError("Configure AZ_VISION_*, AZ_OPENAI_* env vars")

# ---- Helpers: call Azure Vision Read (OCR) and Image Analyze (caption/tags) ----
def vision_read_image_bytes(image_bytes: bytes):
    # Read API (v3.2 or latest) - synchronous simple example
    url = AZ_VISION_ENDPOINT.rstrip("/") + "/vision/v3.2/read/analyze"
    headers = {"Ocp-Apim-Subscription-Key": AZ_VISION_KEY, "Content-Type": "application/octet-stream"}
    r = requests.post(url, headers=headers, data=image_bytes)
    r.raise_for_status()
    # Get operation-location to poll
    op_url = r.headers.get("Operation-Location")
    # poll until done (simple loop, add timeout in prod)
    resp = requests.get(op_url, headers={"Ocp-Apim-Subscription-Key": AZ_VISION_KEY})
    while resp.json().get("status") in ("running", "notStarted"):
        import time; time.sleep(0.5)
        resp = requests.get(op_url, headers={"Ocp-Apim-Subscription-Key": AZ_VISION_KEY})
    # parse lines
    lines = []
    for readResult in resp.json().get("analyzeResult", {}).get("readResults", []):
        for line in readResult.get("lines", []):
            lines.append(line.get("text"))
    return lines

def vision_analyze_image_bytes(image_bytes: bytes):
    # Image Analysis: get caption and tags (Image Analysis 4.0 example)
    url = AZ_VISION_ENDPOINT.rstrip("/") + "/vision/v4.0/analyze?visualFeatures=Categories,Description,Tags,Objects"
    headers = {"Ocp-Apim-Subscription-Key": AZ_VISION_KEY, "Content-Type": "application/octet-stream"}
    r = requests.post(url, headers=headers, data=image_bytes)
    r.raise_for_status()
    j = r.json()
    caption = None
    if "description" in j and j["description"].get("captions"):
        caption = j["description"]["captions"][0]["text"]
    tags = [t["name"] for t in j.get("tags",[])]
    objects = [o["object"] for o in j.get("objects",[])]
    return {"caption": caption, "tags": tags, "objects": objects, "raw": j}

# ---- Prompt engineering: template ----
PROMPT_TEMPLATE = """
You are a security analyst. Input below describes an application architecture diagram (extracted caption, OCR text, detected objects).
Task: Produce a STRIDE threat analysis per identified component.
Output: JSON with keys: 'components': [{ 'name':..., 'evidence':..., 'stride': { 'Spoofing': [...], 'Tampering': [...], 'Repudiation': [...], 'InfoDisclosure': [...], 'DoS': [...], 'Elevation': [...] }, 'recommendations': [...] }]

Diagram metadata:
CAPTION: {caption}
OCR_TEXT: {ocr_text}
DETECTED_OBJECTS: {objects}
ADDITIONAL_NOTES: {notes}

Rules:
- Identify components (e.g., "Load Balancer", "Auth Service", "Database", "API Gateway", "Mobile Client") using the evidence fields.
- For each component, produce at least one threat item per applicable STRIDE category (omit category only if clearly N/A).
- Provide concise mitigation recommendations (3-6 words each).
- Output strictly valid JSON only.
"""

# ---- Azure OpenAI call (ChatCompletion style using openai lib or REST) ----
def call_azure_openai(prompt: str):
    # simple REST call using OpenAI-compatible endpoint (adjust api-version if needed)
    url = AZ_OPENAI_ENDPOINT.rstrip("/") + f"/openai/deployments/{AZ_OPENAI_DEPLOYMENT}/chat/completions?api-version=2023-03-15-preview"
    headers = {"api-key": AZ_OPENAI_KEY, "Content-Type": "application/json"}
    body = {
      "messages":[{"role":"system","content":"You are a security analyst."},
                  {"role":"user","content":prompt}],
      "max_tokens":800,
      "temperature":0.0
    }
    r = requests.post(url, headers=headers, json=body)
    r.raise_for_status()
    resp = r.json()
    # extract assistant content (depends on model response shape)
    return resp["choices"][0]["message"]["content"]

# ---- FastAPI endpoint ----
class AnalyzeResponse(BaseModel):
    caption: str | None
    ocr_text: list
    detected_objects: list
    stride_analysis: dict

@app.post("/analyze", response_model=AnalyzeResponse)
async def analyze_arch_image(file: UploadFile = File(...)):
    data = await file.read()
    try:
        ocr_lines = vision_read_image_bytes(data)
        analysis = vision_analyze_image_bytes(data)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    caption = analysis.get("caption")
    objects = analysis.get("objects",[])
    prompt = PROMPT_TEMPLATE.format(caption=caption or "", ocr_text="\\n".join(ocr_lines), objects=", ".join(objects), notes="Use STRIDE")
    try:
        ai_out = call_azure_openai(prompt)
        # expect JSON string; try parse
        stride_json = json.loads(ai_out)
    except Exception as e:
        # return raw AI output if JSON parsing fails
        stride_json = {"raw_ai_output": ai_out, "parse_error": str(e)}
    return {"caption": caption, "ocr_text": ocr_lines, "detected_objects": objects, "stride_analysis": stride_json}

Notas sobre o exemplo

O código usa Azure Vision Read e Image Analyze via REST; em produção prefira SDKs oficiais (azure-ai-vision, azure-ai-openai) com autenticação robusta.

Ajuste api-version e deployment conforme sua subscrição. A URL/versão de API pode mudar: ver docs oficiais.

O prompt template é a peça central: inclua evidências (nomes de componentes extraídos por OCR ou objetos detectados) para reduzir alucinações.

4 — Dicas de Prompt Engineering (práticas)

Forneça evidência estruturada: caption + OCR + tags em campos separados.

Peça saída estruturada (JSON) e valide o JSON; repetir a instrução “Output strictly valid JSON only.” reduz prob. de formato livre.

Temperatura baixa (0.0–0.2) para análises factuais e consistentes.

Adicionar exemplos (few-shot): inclua 1 exemplo de entrada→JSON STRIDE para ensinar o formato.

Verificação pós-processamento: validar se todas categorias STRIDE existem e adicionar placeholders se ausentes.

5 — Observações, riscos e recomendações técnicas

Risco de hallucination do LLM ao identificar componentes não explicitamente presentes — mitigar: augment prompt with concrete OCR/object evidence e exigir evidence em cada componente.

Privacidade: imagens arquiteturais podem conter segredos; garanta políticas de retenção/encryption e uso de redes privadas (VNet).

Limitações: visão automática pode não reconhecer simbologia customizada; indique ao usuário requisito de mapear símbolos padrão (legenda) ou permita upload de metadados (JSON) junto com imagem.

Validação humana: adicione etapa de revisão manual antes de aplicar mitigação em produção.

6 — Referências (obrigatórias)

Azure OpenAI / Responses & Chat Quickstarts — Microsoft Learn.
Microsoft Learn

Azure Computer Vision — Read (OCR) & Image Analysis quickstarts.
Microsoft Learn

STRIDE threat model — Microsoft / referência STRIDE (definição e categorias).
Microsoft Learn